In [2]:
# Import Libraries
import requests
import json
import datetime
import pandas as pd

# Define Parameters Dictionary
params = dict()
params['access_token'] = 'EAAr1Pr9o9QcBAFUvX1EOg2fUE9dxciszFoZCUjlKaqwQolcXp8b8edwjjobZCX0GMuR45Y1uDZCG7xWCTUNz0LBPSBbaQ8ygVwVD3BHwDZBH88R39ZANdoFuSOhJhluEYgSyTV3JVYit07z0HBnRRnRNQ3BvGkUUyZBJSPb5KGT6BNnH92EMDg7BGli9R3hFEZD'        # TOKEN DE ACESSO
params['client_id'] = '3084399615210759'                  # not an actual client id
params['client_secret'] = '199bcd728b98322cec2dd66cfc9ffa5a'     # not an actual client secret
params['graph_domain'] = 'https://graph.facebook.com'
params['graph_version'] = 'v12.0'
params['endpoint_base'] = params['graph_domain'] + '/' + params['graph_version'] + '/'
params['page_id'] = '106152182044947'                  # not an actual page id
params['instagram_account_id'] = '17841401411209088'        # not an actual instagram business account id
params['ig_username'] = 'viniciusvianavieira'

In [3]:
endpointParams = dict()
endpointParams['input_token'] = params['access_token']
endpointParams['access_token'] = params['access_token']

# Define URL
url = params['graph_domain'] + '/debug_token'


# Requests Data
data = requests.get(url, endpointParams)
access_token_data = json.loads(data.content)
# access_token_data

In [4]:
print("Token Expires: ", datetime.datetime.fromtimestamp(access_token_data['data']['expires_at']))


Token Expires:  2022-05-27 15:04:10


In [5]:
# Define URL
url = params['endpoint_base'] + 'oauth/access_token'

# Define Endpoint Parameters
endpointParams = dict() 
endpointParams['grant_type'] = 'fb_exchange_token'
endpointParams['client_id'] = params['client_id']
endpointParams['client_secret'] = params['client_secret']
endpointParams['fb_exchange_token'] = params['access_token']


# Requests Data
data = requests.get(url, endpointParams )
long_lived_token = json.loads(data.content)
#long_lived_token




In [8]:
# Define URL
url = params['endpoint_base'] + params['instagram_account_id'] + '/media'


# Define Endpoint Parameters
endpointParams = dict()
endpointParams['fields'] = 'id,caption'#,media_product_type,media_type,permalink,thumbnail_url,timestamp,username,like_count,comments_count#'
endpointParams['access_token'] = params['access_token']

# Requests Data
data = requests.get(url, endpointParams )
basic_insight = json.loads(data.content)

fotos_ids = []
for i in range(0,len(basic_insight['data'])):
    fotos_ids.append(basic_insight['data'][i]['id'])


Existe_proxima_pagina = True
while Existe_proxima_pagina:
    try:
        url_next = basic_insight['paging']['next']
        data_next = requests.get(url_next)
        basic_insight = json.loads(data_next.content)
        for i in range(0,len(basic_insight['data'])):
            fotos_ids.append(basic_insight['data'][i]['id'])
    except Exception as e:
        Existe_proxima_pagina = False
        break 
print(fotos_ids)
len(fotos_ids)

{'fields': 'id,caption'}
['17942106577637897', '17904444272084256', '17942783185461876', '17863822130361776', '17859228353363822', '17858653037118951', '18043185997250377', '17852092691034198', '17848915328038738', '17852305657916509', '17853626338881791', '17858301031774359', '17857926820694928', '18069295828197368', '17880700216459824', '17908776652349533', '18077893972056002', '18052256560118502', '18062653915015064', '17992208503214024', '18007770499098386', '18005044594115059', '17901002038301661', '17986760833110266', '17924048794186036', '17848927153268949', '17913890452166302', '17934698071027214', '17897646331193087', '17900424589092720', '17843380978236427', '17887556497164429', '17904466954068878', '17909851345046577', '17891828497096669', '17879278012136110', '17884851775108984', '17894789221013045', '17865803515176363', '17867657392088420', '17879867746008420', '17872511521012208', '17872597360028041', '17871193348052829', '17857456954092670', '17856607219080778', '1786474

56

In [7]:

df = pd.DataFrame(basic_insight['data'])
df.columns = ['id', 'Caption', 'Média_Product_Type','Media_Type', 'Permalink', 'Timestamp', 'Username', 'Likes', 'Comments','Nothing']
df

ValueError: Length mismatch: Expected axis has 2 elements, new values have 10 elements

In [ ]:
#SEPARA OS ID'S E ADICIONA EM UMA LISTA
id_fotos_list = []
for i in range(0,(len(basic_insight['data']))):
    id_fotos_list.append(basic_insight['data'][i]['id'])

#TRANSFORMA ESSA LISTA EM UM DICIONARIO
parametros = {'id_fotos':id_fotos_list}


# Define Endpoint Parameters



# Define URL
fotos_insight = []
for i in range(0,len(parametros['id_fotos'])):
    url = params['endpoint_base'] + parametros['id_fotos'][i]+ '/insights'


    endpointParams1 = dict()
    endpointParams1['metric'] = 'engagement,impressions,reach,saved,video_views'
    endpointParams1['access_token'] = params['access_token']
   
    try:
         data = requests.get(url, endpointParams1 )
         fotos_basic_insight = json.loads(data.content)
         print(fotos_basic_insight)
         fotos_insight.append(fotos_basic_insight['data'])
    except:
         pass

    # print(fotos_insight)

     





{'data': [{'name': 'engagement', 'period': 'lifetime', 'values': [{'value': 196}], 'title': 'Interação', 'description': 'Total de gostos e comentários no conteúdo', 'id': '17942106577637897/insights/engagement/lifetime'}, {'name': 'impressions', 'period': 'lifetime', 'values': [{'value': 1361}], 'title': 'Impressões', 'description': 'Total de vezes que o conteúdo foi visto', 'id': '17942106577637897/insights/impressions/lifetime'}, {'name': 'reach', 'period': 'lifetime', 'values': [{'value': 983}], 'title': 'Alcance', 'description': 'Total de contas individuais que viram o conteúdo', 'id': '17942106577637897/insights/reach/lifetime'}, {'name': 'saved', 'period': 'lifetime', 'values': [{'value': 0}], 'title': 'Guardado', 'description': 'Total de contas individuais que guardaram este conteúdo', 'id': '17942106577637897/insights/saved/lifetime'}, {'name': 'video_views', 'period': 'lifetime', 'values': [{'value': 0}], 'title': 'Visualizações do vídeo', 'description': 'Total de vezes que o 

In [ ]:
media_insight = []

# Loop Over 'Media ID'
for i in basic_insight['data']:
    params['latest_media_id'] = i['id']
   
    # Define URL
    url = params['endpoint_base'] + params['latest_media_id'] + '/insights'

    # Define Endpoint Parameters
    endpointParams = dict() 
    endpointParams['metric'] = 'engagement,impressions,reach,saved,video_views'
    endpointParams['access_token'] = params['access_token'] 
    
    # Requests Data
    data = requests.get(url, endpointParams )
    json_data_temp = json.loads(data.content)
  
    try:
        media_insight.append(list(json_data_temp['data']))

    except:
        pass
    

In [ ]:

# Initialize Empty Container
engagement_list = []
impressions_list = []
reach_list = []
saved_list = []
video_views = []

# Loop Over Insights to Fill Container
for insight in media_insight:
    engagement_list.append(insight[0]['values'][0]['value'])
    impressions_list.append(insight[1]['values'][0]['value'])
    reach_list.append(insight[2]['values'][0]['value'])
    saved_list.append(insight[3]['values'][0]['value'])
    video_views.append(insight[4]['values'][0]['value'])
    
# Create DataFrame
df_media_insight = pd.DataFrame(list(zip(engagement_list, impressions_list, reach_list, saved_list,video_views)), columns =['Engagement', 'Impressions', 'Reach', 'Saved','Views in videos'])
df_media_insight.head()

,Engagement,Impressions,Reach,Saved,Views in videos
0,196,1361,983,0,0
1,330,3363,2011,0,0
2,374,3299,2006,0,0
3,242,2627,1744,0,0
